## Chi-Squared Analysis

### First Question: Did more women and children survive compared to men?

Dataset from Canvas (Titanic Passenger List.csv)

In [ ]:
import pandas as pd

passengers_df = pd.read_csv('../data/Titanic_Passengers.csv')

passengers_df.head(20)

In [ ]:
passengers_df.tail(20)

- What kind of data is the `dependent` variable?
- What kind of data is the `independent` variable?
    - Nominal

Two Types:
    - `Ordinal` data. Data that can be ordered
    - `Categorical/nominal`. Data that can be ordered

In [ ]:
# Margins = True will show the summary
contingency_table = pd.crosstab(passengers_df['Gender'], passengers_df['Lived_Died'], margins=True)
print(contingency_table)

In [ ]:
# Task: Create a stacked bar chart showing the number of passenger by gender that lived or died.

import matplotlib.pyplot as plt

#Exclude the last column and row (summary info)

# For plotting do not include margins=True
contingency_table = pd.crosstab(passengers_df['Gender'], passengers_df['Lived_Died'])
# contingency_table.iloc[:-1,:-1].plot(kind='bar', stacked=True)
contingency_table.plot(kind='bar', stacked=True)
plt.ylabel('Count')

In [ ]:
from scipy import stats # for chi_contingency

print(stats.chi2_contingency(contingency_table)[0:3])

# you get 3 values back
# 1) Chi_Squared: The higher the chi_squared statistics, the less likelihood that the observed values were the result of pure chance 
# 2) Target P-Value
# 3) Degrees of freedom: 1 degree of freedom. (rows -1) * (r columns -1) = 1

## Chi-Square Example: Was Age Relevant?

In [ ]:
# Check the age column. It is already fixed for the age at the time of the incident

# What about null values?

# Create the bins
#passengers_df['age_bins'] = pd.cut(x=passengers_df['Age'], bins=[0, 10, 20, 30, 40, 50, 60, 70, 80])


passengers_df['age_bins'] = pd.cut(x=passengers_df['Age'], bins=range(0,90,10))

contingency_table = pd.crosstab(passengers_df['age_bins'], passengers_df['Lived_Died'])
print(contingency_table)
print(f'Chi2-Stats {stats.chi2_contingency(contingency_table)[0:3]}')

#contingency_table.plot(kind='hist')


In [ ]:
passengers_df.groupby(['age_bins'], ['Lived_Died']).size().unstack().plot(kind='bar', stacked=False)
plt.ylabel('Count')
plt.xlabel('Age Ranges')
plt.title('Histogram of Age Ranges')

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

# Load data
passengers_df = pd.read_csv('../data/Titanic_Passengers.csv')

passengers_df
# Create a heatmap
passengers_df['age_bins'] = pd.cut(x=passengers_df['Age'], bins=[0, 10, 20, 30, 40, 50, 60, 70, 80])

# Use a pivottable

## EKSIK

In [ ]:
# Drop columns that are not needed
passengers_df.drop(['Name', 'Born', 'Died', 'Fare', 'Fare_decimal', 'Passenger_class'], axis=1, inplace=True)
passengers_df.dropna(inplace=True) # drop null values

#Encode the categorical data
le = preprocessing.LabelEncoder()
passengers_df['Lived_Died_int'] =le.fit_transform(passengers_df['Lived_Died'])
passengers_df['Age_Groups'] = pd.cut(x=passengers_df['Age'], bins=range(0,90,10)) # create bins
pivot_table = pd.pivot_table(passengers_df, values='Lived_Died_int', index=['Gender'], columns=['Age_Groups']) # Use a pivottable
pivot_table.applymap(lambda)

##EKSIK

## Task: Did the Titanic Passenger Class Affected Survivorship
In this case, the data is `ordinal` because we can order the categories from first to third

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from scipy import stats

# Load data
passengers_df = pd.read_csv('../data/Titanic_Passengers.csv')

passengers_df.head()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(passengers_df['Passenger_Class'], passengers_df['Lived_Died'], margins=True)
contingency_table

In [ ]:
# Create a Bar Plot with the data stacked
contingency_table = pd.crosstab(passengers_df['Lived_Died'],passengers_df['Passengers_Class'], margins=False)
contingency_table.plot(kind='bar', stacked=True)

In [ ]:
# Run a chi2_contingency on the table
contingency_table = pd.crosstab(passengers_df['Passenger_Class'], passengers_df['Lived_Died'], margins=False)
stats.chi2_contingency(contingency_table)[0:3]



Which passenger class is responsible for the significant difference in survival rate?
- Chi-square test offers that at least `one` of the passenger class is different from what we would expect if the variables were independent
- Since this is our first test, it is considered a `Priori Test`
- To continue our analysis, we will run another test, a `Post Hoc` test


### Post-Hoc Test (Fisher's Exact Test)
- Compared every passenger class to every other class to see if there is a statistical significance between the two.
- We could apply a `Chi-Square` test for each of the three separate comparison, but our sample data is too small.
- In this case, `Fisher's Exact` test is a better option.

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from scipy import stats

# Load data
passengers_df = pd.read_csv('../data/Titanic_Passengers.csv')
contingency_table = pd.crosstab(passengers_df['Passengers_Class'], passengers_df['Lived_Died'],margins=False)
contingency_table

In [ ]:
first_class = contingency_table.iloc[0]. values # extract row values
second_class = contingency_table.iloc[1]. values # extract row values
third_class = contingency_table.iloc[2]. values # extract row values
# Create a Fisher's Exact test to determine if there is a relationship between passenger classes and survival
oddsratio, pvalue = stats.fisher_exact([first_class], [second_class])
print(f'Fishers Exact Test for First and Second class: {pvalue}')
oddsratio, pvalue = stats.fisher_exact([first_class], [third_class])
print(f'Fishers Exact Test for First and Third class: {pvalue}')
oddsratio, pvalue = stats.fisher_exact([second_class], [third_class])
print(f'Fishers Exact Test for Second and Third class: {pvalue}')

Each of the Fisher's Test reveals statistical significance

Another way to conceptualize the result is to look at the percentages:

| Passenger Class | Died |
|-----------------|------|
| First Class     | 43%  |
| Second Class    | 60%  |
| Third Class     | 74%  |

TODO: Create a heatmap of Passenger Class split in Age_Ranges by Gender and Survival rate

In [ ]:
# Drop columns that are not needed
passengers_df.drop(['Name', 'Born', 'Died', 'Fare', 'Fare_decimal', 'Passenger_class'], axis=1, inplace=True)
passengers_df.dropna(inplace=True) # drop null values

#Encode the categorical data
le = preprocessing.LabelEncoder()
passengers_df['Lived_Died_int'] =le.fit_transform(passengers_df['Lived_Died'])
passengers_df['Age_Groups'] = pd.cut(x=passengers_df['Age'], bins=range(0,90,10)) # create bins
pivot_table = pd.pivot_table(passengers_df, values='Lived_Died_int', index=['Gender'], columns=['Age_Groups']) # Use a pivottable
pivot_table.applymap(lambda x: 1 - x) # invert values in heatmap

# Create a heatmap
plt.figure(figsize=(16,5))
x_axis_labels = [

]
sns.heatmap(pivot_table, annot=True, yticklabels=['Female', 'Male'], xticklabels=)


## EKSIK


### T-Test: Fare and Gender

While `Chi-Test` and `Fisher-Test` focus on `categorical` discrete dependent variable, the `T-Test` and `ANOVA` test, target `continuous` data

#### How was fare (price of passenger) Affected by Gender and Passenger Class?
- Did one gender pay more than the other?
- Is there a difference in fare between passenger class?
- Is there an interaction between gender and passenger class on the resulting fare?

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from scipy import stats

# Load data
passengers_df = pd.read_csv('../data/Titanic_Passengers.csv')

In [ ]:
# Visualize data using boxplot
sns.boxplot(y = passengers_df['Fare decimal'], x=passengers_df['Gender'])
plt.ylabel('Fare Amount in Pounds')

# EKSIK


In [ ]:
male = passengers_df[passengers_df['Gender'] == 'Male']
print = (f'Male Summary {male['Fare_decimal'].describe()}')

In [ ]:
female = passengers_df[passengers_df['Gender'] == 'Female']
print = (f'Female Summary {female['Fare_decimal'].describe()}')

In [ ]:
from scipy.stats import ttest_ind
# Drop null values
passengers_df.dropna(inplace=True)
male = passengers_df[passengers_df['Gender'] == 'Male']
female = passengers_df[passengers_df['Gender'] == 'Female']

#Run test
statistics, pvalue = ttest_ind(male['Fare_decimal'], female['Fare_decimal'])
print(f'T-test for Fare Paid p_value {pvalue}')
